In [1]:
import pandas as pd
import numpy as np
import ast
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
from tqdm import tqdm
import warnings
import json

In [14]:
warnings.filterwarnings('ignore')

In [47]:
knn_ranker_test = pd.read_csv('initial_test_ranker.csv')
knn_ranker_test.head()

,user_id,vacancy_id,vacancy_id_preds,index_test,vacancy_id_true
0,u_263456,v_1900367,"['v_1900367', 'v_1915162', 'v_1514089', 'v_268...",v_1900367,"['v_1711113', 'v_2501360', 'v_141541', 'v_4846..."
1,u_263456,v_1900367,"['v_1900367', 'v_1915162', 'v_1514089', 'v_268...",v_2678071,"['v_1903334', 'v_2566704', 'v_1775258', 'v_829..."
2,u_263456,v_1900367,"['v_1900367', 'v_1915162', 'v_1514089', 'v_268...",v_459840,"['v_580765', 'v_2397911', 'v_1952634', 'v_2255..."
3,u_263456,v_1900367,"['v_1900367', 'v_1915162', 'v_1514089', 'v_268...",v_1078795,"['v_1900367', 'v_1844848', 'v_703243', 'v_2094..."
4,u_263456,v_1900367,"['v_1900367', 'v_1915162', 'v_1514089', 'v_268...",v_2142328,"['v_2443974', 'v_464599', 'v_767341', 'v_19097..."


In [49]:
knn_ranker_test['vacancy_id_true'] = knn_ranker_test['vacancy_id_true']\
                                .apply(lambda x: ast.literal_eval(x))\
                                .apply(lambda x: [id.strip(" '[]") for id in x])
knn_ranker_test['vacancy_id_preds'] = knn_ranker_test['vacancy_id_preds']\
                                .apply(lambda x: ast.literal_eval(x))\
                                .apply(lambda x: [id.strip(" '[]") for id in x])

In [11]:
vacancy = pd.read_csv('vacancies_full_keyskills.csv')
vacancy.dropna(subset=['keySkills_str'], inplace=True)
vacancy.head()

,vacancy_id,name,company_id,keySkills,compensation_from,compensation_to,compensation_currencyCode,area_id,area_regionId,employment,workSchedule,workExperience,clean_name,area_regionId_encoder,area_id_encoder,company_id_encoder,employment_encoder,workSchedule_encoder,workExperience_encoder,keySkills_str
0,v_862116,Смотритель музейный,c_162972,"['Уверенный пользователь пк', 'Работа в команд...",16500.0,NaN,RUR,a_4761,ar_33,full,fullDay,noExperience,Смотритель музейный,32,4181,69972,5,5,1,Уверенный пользователь пк Работа в команде Уме...
1,v_288642,Ведущий менеджер по работе с физическими лицами,c_208672,"['Активные продажи', 'Холодные продажи', 'Кред...",50000.0,NaN,RUR,a_744,ar_2,full,fullDay,noExperience,Ведущий менеджер по работе с физическими лицами,17,6731,120750,5,5,1,Активные продажи Холодные продажи Кредитные до...
2,v_312507,Торговый представитель (г. Абакан),c_206699,"['Продуктивность', 'Клиентоориентированность',...",60000.0,NaN,RUR,a_6837,ar_4,full,fullDay,between1And3,Торговый представитель,39,6487,118557,5,5,2,Продуктивность Клиентоориентированность Работа...
3,v_2323706,Менеджер по качеству,c_244141,"['Контроль качества', 'Производственный контро...",88000.0,NaN,RUR,a_5920,ar_64,full,fullDay,between1And3,Менеджер по качеству,66,5469,160161,5,5,2,Контроль качества Производственный контроль На...
4,v_704266,Менеджер по продажам / Sales manager,c_192038,"['Активные продажи', 'Поиск и привлечение клие...",30000.0,100000.0,RUR,a_5398,ar_71,full,fullDay,between1And3,Менеджер по продажам,74,4888,102268,5,5,2,Активные продажи Поиск и привлечение клиентов ...


In [74]:
%run '../metric/metric_mrr'

In [79]:
knn_ranker_test_sample = knn_ranker_test.sample(n=100).copy()

In [80]:
tqdm.pandas() 

knn_ranker_test_sample['mrr'] = knn_ranker_test_sample.progress_apply(lambda row:
                                                                      calculate_mrr(row['vacancy_id_true'],
                                                                                    row['vacancy_id_preds'],
                                                                                    vacancy,
                                                                                    threshold=0.75), axis=1)

100%|█████████████████████████████████████████| 100/100 [06:16<00:00,  3.77s/it]


In [97]:
knn_ranker_test_sample.head()

,user_id,vacancy_id,vacancy_id_preds,index_test,vacancy_id_true,mrr
840,u_1075065,v_220346,"[v_22147, v_1766831, v_1682712, v_1886127, v_2...",v_220346,"[v_2658628, v_1293659, v_1130752, v_1474824, v...",1.000000
854,u_496550,v_2372014,"[v_2372014, v_2141281, v_364804, v_212598, v_2...",v_2372014,"[v_1685666, v_480866, v_423024, v_1860459, v_1...",0.833333
625,u_1172759,v_2632068,"[v_2632068, v_601754, v_790273, v_2240840, v_5...",v_2632068,"[v_1836899, v_332650, v_1860505, v_2616925, v_...",1.000000
592,u_394374,v_1888988,"[v_1824415, v_643121, v_539808, v_1911870, v_2...",v_1888988,"[v_1404833, v_616130, v_2059138, v_2093006, v_...",0.125000
236,u_229008,v_2459924,"[v_2459924, v_1847717, v_521972, v_1026606, v_...",v_2459924,"[v_1234605, v_2603303, v_578551, v_2355879, v_...",0.733333


In [82]:
knn_ranker_test_sample['mrr'].median()

0.7

In [91]:
second_ranker_test = pd.read_csv('second_test_ranker.csv')

In [92]:
second_ranker_test['vacancy_id_true'] = second_ranker_test['vacancy_id_true']\
                                .apply(lambda x: ast.literal_eval(x))\
                                .apply(lambda x: [id.strip(" '[]") for id in x])
second_ranker_test['vacancy_id_preds'] = second_ranker_test['vacancy_id_preds']\
                                .apply(lambda x: ast.literal_eval(x))\
                                .apply(lambda x: [id.strip(" '[]") for id in x])

In [93]:
second_ranker_test_sample = second_ranker_test.sample(n=100).copy()

In [115]:
tqdm.pandas() 

second_ranker_test_sample['mrr'] = second_ranker_test_sample.progress_apply(lambda row:
                                                                      calculate_mrr(row['vacancy_id_true'],
                                                                                    row['vacancy_id_preds'],
                                                                                    vacancy,
                                                                                    threshold=0.75), axis=1)

100%|█████████████████████████████████████████| 100/100 [05:44<00:00,  3.45s/it]


In [119]:
second_ranker_test_sample.head()

,user_id,vacancy_id,vacancy_id_preds,index_test,vacancy_id_true,mrr
662,u_513845,v_463006,"[v_463006, v_1969855, v_2203768, v_2415240, v_...",v_463006,"[v_1190877, v_1075562, v_569583, v_679105, v_1...",1.000000
850,u_316820,v_1933442,"[v_469763, v_1933442, v_1289507, v_1735561, v_...",v_1933442,"[v_1196591, v_1018862, v_413854, v_2607750, v_...",0.352667
886,u_581971,v_1008293,"[v_1795950, v_325366, v_1596742, v_2420819, v_...",v_1008293,"[v_690837, v_1722021, v_125867, v_1317901, v_1...",0.402500
26,u_884002,v_1386149,"[v_2379686, v_1432624, v_705573, v_2433013, v_...",v_2656775,"[v_2717209, v_48074, v_441875, v_2449676, v_13...",0.472105
921,u_992318,v_350947,"[v_640272, v_257629, v_1268792, v_1250589, v_9...",v_350947,"[v_189320, v_2298305, v_473797, v_2675697, v_2...",0.952857


In [120]:
second_ranker_test_sample['mrr'].median()

0.8049999999999999